article class = sc-1ww42at-0, EyLft qjwil1-0 iCnXma

In [124]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import folium
import branca.colormap as cm
pd.set_option('display.max_columns', None)

In [3]:
# send requests

url = 'https://www.immoscout24.ch/de/wohnung/mieten/ort-luzern?pn=1&r=6'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

In [10]:
# read out table
raw_data = soup.find(id='state')

jason_data = json.loads(raw_data.string[25:])
property_list = jason_data['pages']['searchResult']['resultData']['allProperties']
parsed_list = []

for elem in property_list:
    price =  elem['priceFormatted']['value']
    parsed_list.append([elem['id'], price, elem['latitude'], elem['longitude']])

#parsed_list

In [38]:
property_df = pd.DataFrame(parsed_list)
property_df.columns = ['Id', 'Price', 'Latitude', 'Longitude']

filter = property_df['Price'] != 'Preis auf Anfrage'
property_df = property_df[filter]

property_df['Price'] = property_df['Price'].str[4:-2]
property_df['Price'] = property_df['Price'].str.replace('\'', '')
property_df['Price'] = property_df['Price'].astype('int64')

property_df.describe()

,Id,Price,Latitude,Longitude
count,6.990000e+02,699.000000,699.000000,699.000000
mean,5.762488e+06,1910.690987,47.053289,8.302386
std,4.584244e+05,783.756023,0.019668,0.026927
min,6.020960e+05,130.000000,46.994750,8.230550
25%,5.795507e+06,1490.000000,47.038370,8.282420
50%,5.914668e+06,1740.000000,47.052210,8.301000
75%,5.948961e+06,2165.000000,47.068040,8.313595
max,5.972088e+06,7900.000000,47.097020,8.380820


In [123]:
# send requests
result_list = []
max = 30

for i in range(30):
    url = 'https://www.immoscout24.ch/de/wohnung/mieten/ort-luzern?pn={}&r=6'.format(i+1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    raw_data = soup.find(id='state')
    jason_data = json.loads(raw_data.string[25:])
    data_list = jason_data['pages']['searchResult']['resultData']['listData']

    page_list = []

    for elem in data_list:
        if 'price' in elem: price = elem['price']
        else: price = np.nan

        if 'surfaceLiving' in elem: sl = elem['surfaceLiving']
        else: sl = np.nan

        if 'numberOfRooms' in elem: nor = elem['numberOfRooms']
        else: nor = np.nan

        if 'latitude' in elem: lat = elem['latitude']
        else: lat = np.nan

        if 'longitude' in elem: lon = elem['longitude']
        else: lon = np.nan

        page_list.append([elem['id'], price, nor, sl, elem['zip'], lat, lon])

    result_list += page_list

#result_list

In [47]:
result_df = pd.DataFrame(result_list)
result_df.columns = ['Id', 'Price', 'RoomNr', 'Surface', 'ZIP', 'Latitude', 'Longitude']

result_df.dropna(inplace=True)
result_df.describe()

,Id,Price,RoomNr,Surface,Latitude,Longitude
count,6.050000e+02,605.000000,605.000000,605.00000,605.000000,605.000000
mean,5.754599e+06,1962.897521,3.445455,84.33719,47.053277,8.302904
std,4.743571e+05,794.792415,1.052591,31.33353,0.019901,0.027130
min,6.020960e+05,170.000000,1.000000,1.00000,46.994750,8.251420
25%,5.775736e+06,1530.000000,3.000000,66.00000,47.038240,8.281830
50%,5.909135e+06,1790.000000,3.500000,83.00000,47.052210,8.301570
75%,5.947662e+06,2210.000000,4.500000,101.00000,47.069150,8.314500
max,5.973348e+06,7900.000000,7.000000,234.00000,47.097020,8.380820


In [48]:
# create map of lucerne using latitude and longitude values
map_lucerne = folium.Map(location=[47.05, 8.3], zoom_start=12.5)

linear = cm.LinearColormap(
    ['green', 'red'],
    vmin=100, vmax=8000
)

linear

# add markers to map
for lat, lng, price in zip(result_df['Latitude'], result_df['Longitude'], result_df['Price']):
    label = folium.Popup(str(price), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=linear(price),
        fill=True,
        fill_color=linear(price),
        fill_opacity=0.7,
        parse_html=False).add_to(map_lucerne)

map_lucerne  

In [135]:
CLIENT_ID = 'VV5H111QRCSXOLD5LF52CDDBI1S1QPDX5BFBOI3IJQHGYI4L' # your Foursquare ID
CLIENT_SECRET = 'TEH4SQMZBUB1VHCJZQYNLNHCQTBLTEXIHBLY1VWM1FMSYU1N' # your Foursquare Secret
VERSION = '20180604'
RADIUS = 500
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

flatID = 8015
latitude = 47.04751
longitude = 8.30613

#47.04872, 8.29963

Your credentails:
CLIENT_ID: VV5H111QRCSXOLD5LF52CDDBI1S1QPDX5BFBOI3IJQHGYI4L
CLIENT_SECRET:TEH4SQMZBUB1VHCJZQYNLNHCQTBLTEXIHBLY1VWM1FMSYU1N


In [132]:
sample = result_df.head(100)
sample

,Id,Price,RoomNr,Surface,ZIP,Latitude,Longitude
0,5953070,1680.0,3.0,77.0,6003,47.05248,8.29721
1,5961802,1510.0,2.5,52.0,6006,47.05984,8.30959
2,5961175,2015.0,3.5,101.0,6020,47.07086,8.25988
3,4548088,1400.0,3.5,67.0,6020,47.08441,8.26981
4,5963173,1675.0,2.5,48.0,6014,47.06496,8.28089
...,...,...,...,...,...,...,...
128,5889889,1390.0,3.0,64.0,6004,47.05640,8.29001
129,5904184,1390.0,2.5,55.0,6004,47.06034,8.30889
130,5921178,1390.0,3.5,67.0,6020,47.08441,8.26981
133,5876143,1400.0,3.5,62.0,6010,47.03509,8.28414


In [136]:

venues_list=[]

for flatID, flatLat, flatLon in zip(sample['Id'], sample['Latitude'], sample['Longitude']):

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, flatLat, flatLon, VERSION, RADIUS, LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']
    for v in results:
        venues_list.append([
            flatID,
            v['venue']['categories'][0]['name']
        ])

venues_df = pd.DataFrame(venues_list)
venues_df.columns = ['ID', 'Category']

onehot = pd.get_dummies(venues_df['Category'],  prefix="", prefix_sep="")
onehot['ID'] = venues_df['ID']
onehot = onehot.groupby('ID').sum()

In [137]:
onehot['sum'] = onehot.sum(axis=1)
onehot

,Art Museum,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bakery,Bar,Beach,Beer Store,Bistro,Boat or Ferry,Brewery,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cable Car,Café,Candy Store,Casino,Chinese Restaurant,Chocolate Shop,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cultural Center,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pizza Place,Plaza,Pool,Pub,Restaurant,Rock Club,Scenic Lookout,Shopping Mall,Soccer Field,Soccer Stadium,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Water Park,Wine Shop,sum
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2218220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3875847,0,1,0,0,0,0,0,0,3,0,0,0,0,0,0,2,0,0,1,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,24
4400099,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,9
4548088,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
4571524,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970067,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,13
5971340,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,